In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50, Xception, InceptionV3
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

# Define LeNet-5
def create_lenet5(input_shape):
    model = Sequential([
        Conv2D(6, (5, 5), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(16, (5, 5), activation='relu', padding='valid'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# Define AlexNet
def create_alexnet(input_shape):
    model = Sequential([
        Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        MaxPooling2D((3, 3), strides=2),
        Conv2D(256, (5, 5), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=2),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Define GoogLeNet (InceptionV3)
def create_googlenet(input_shape):
    base_model = InceptionV3(weights='imagenet', input_shape=input_shape, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Define VGGNet (VGG16)
def create_vgg16(input_shape):
    base_model = VGG16(weights='imagenet', input_shape=input_shape, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Define ResNet50
def create_resnet50(input_shape):
    base_model = ResNet50(weights='imagenet', input_shape=input_shape, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Define Xception
def create_xception(input_shape):
    base_model = Xception(weights='imagenet', input_shape=input_shape, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Define SENet (using a simplified approach)
def create_senet(input_shape):
    base_model = Xception(weights='imagenet', input_shape=input_shape, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Load and preprocess MNIST data
def preprocess_image(image, target_size):
    image = tf.image.resize(image, target_size)
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0
    return image

def load_and_preprocess_data(target_size, batch_size=32):
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    train_images = np.expand_dims(train_images, axis=-1)
    test_images = np.expand_dims(test_images, axis=-1)

    # Create TensorFlow dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

    # Preprocess the images
    train_dataset = train_dataset.map(lambda x, y: (preprocess_image(x, target_size), y))
    test_dataset = test_dataset.map(lambda x, y: (preprocess_image(x, target_size), y))

    # Batch and shuffle
    train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)
    
    # One-hot encode labels
    def one_hot_encode(images, labels):
        labels = tf.one_hot(labels, 10)
        return images, labels

    train_dataset = train_dataset.map(one_hot_encode)
    test_dataset = test_dataset.map(one_hot_encode)

    return train_dataset, test_dataset

# Train and evaluate model
def train_and_evaluate_model(model, train_dataset, test_dataset, epochs=10):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Callbacks for early stopping and model checkpoint
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
    ]
    
    # Print training progress
    class PrintEpochProgress(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            print(f"Epoch {epoch + 1}/{self.params['epochs']}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}")

    history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset, callbacks=callbacks + [PrintEpochProgress()])
    loss, accuracy = model.evaluate(test_dataset, verbose=0)
    return history, loss, accuracy

# Example usage
model_params = {
    'LeNet5': (create_lenet5, (32, 32, 3)),
    'AlexNet': (create_alexnet, (227, 227, 3)),
    'GoogLeNet': (create_googlenet, (299, 299, 3)),
    'VGG16': (create_vgg16, (224, 224, 3)),
    'ResNet50': (create_resnet50, (224, 224, 3)),
    'Xception': (create_xception, (299, 299, 3)),
    'SENet': (create_senet, (224, 224, 3))
}

for model_name, (create_model_fn, input_shape) in model_params.items():
    print(f"\nTraining {model_name}...")
    train_dataset, test_dataset = load_and_preprocess_data(target_size=input_shape[:2])
    model = create_model_fn(input_shape)
    history, loss, accuracy = train_and_evaluate_model(model, train_dataset, test_dataset, epochs=10)
    print(f"{model_name} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")



Training LeNet5...
Epoch 1/10
1875/1875 [==============================] - 47s 24ms/step - loss: 0.1483 - accuracy: 0.9543 - val_loss: 0.0630 - val_accuracy: 0.9797
Epoch 2/10


C:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1875/1875 [==============================] - 56s 30ms/step - loss: 0.0546 - accuracy: 0.9832 - val_loss: 0.0482 - val_accuracy: 0.9850
Epoch 3/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0393 - accuracy: 0.9880 - val_loss: 0.0378 - val_accuracy: 0.9873
Epoch 4/10
1875/1875 [==============================] - 44s 24ms/step - loss: 0.0311 - accuracy: 0.9902 - val_loss: 0.0462 - val_accuracy: 0.9875
Epoch 5/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0258 - accuracy: 0.9917 - val_loss: 0.0501 - val_accuracy: 0.9847
Epoch 6/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0211 - accuracy: 0.9933 - val_loss: 0.0367 - val_accuracy: 0.9895
Epoch 7/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0182 - accuracy: 0.9944 - val_loss: 0.0324 - val_accuracy: 0.9913
Epoch 8/10
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 0.0456 - val_accura